# Install Dependencies

In [ ]:
%%capture
!pip install monai
!pip install dicom2nifti

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/preprocess/final_preprocess.py /content
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/utilities/final_utilities.py /content
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/model/get_model.py /content

In [ ]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss

import torch
from final_utilities import train
from get_model import get_model
from final_preprocess import preprocess_image, prepare

import numpy as np
from skimage import exposure
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Resized,
    ToTensord,
)
from monai.data import CacheDataset, Dataset, DataLoader
from glob import glob
import os
from monai.utils import set_determinism

## Checking for CUDA

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. You can use the GPU.")
else:
    print("CUDA is not available. You can only use the CPU.")

CUDA is available. You can use the GPU.


## Preprocess Dataset

In [ ]:
data_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/dataset/dataset-007/Data_Train_Test/'

In [ ]:
data_in = prepare(data_dir, cache=True)

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:111: FutureWarning: <class 'monai.transforms.utility.dictionary.AddChanneld'>: Class `AddChanneld` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead with `channel_dim='no_channel'`.
  warn_deprecated(obj, msg, warning_category)
/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.transforms.utility.dictionary EnsureChannelFirstd.__init__:meta_keys: Argument `meta_keys` has been deprecated since version 0.9. not needed if ima

# Training

In [ ]:
model_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/results/dataset-007/v3batch'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args = {
    'model_name': 'UNet',
    'pretrained': True,
    'dropout': 0.1
}
model = get_model(args)
model = model.to(device)

# loss_function = nn.CrossEntropyLoss().to(device)
# loss_function = TverskyLoss(alpha=0.7, beta=0.3).to(device)
loss_function = DiceLoss(to_onehot_y=True, softmax=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5, amsgrad=True)

if __name__ == '__main__':
    train(model, data_in, loss_function, optimizer, 150, model_dir)

Streaming output truncated to the last 5000 lines.
221/222, Train_loss: 0.4325
Train_dice: 0.5590
222/222, Train_loss: 0.0677
Train_dice: 0.9263
--------------------
Train loss standard deviation: 0.1134
Train metric standard deviation: 0.1173
Epoch_loss: 0.1252
Epoch_metric: 0.8753
test_loss_epoch: 0.2529
test_dice_epoch: 0.7646
Confusion Matrix:
[[58316019    47515]
 [  211431   145291]]
test_loss_epoch: 0.0045
test_dice_epoch: 0.0137
current epoch: 22 current mean dice: 0.7919
best mean dice: 0.0138 at epoch: 21
----------
epoch 23/150
1/222, Train_loss: 0.0693
Train_dice: 0.9345
2/222, Train_loss: 0.1102
Train_dice: 0.9027
3/222, Train_loss: 0.0921
Train_dice: 0.9229
4/222, Train_loss: 0.0567
Train_dice: 0.9483
5/222, Train_loss: 0.1031
Train_dice: 0.9128
6/222, Train_loss: 0.0886
Train_dice: 0.9229
7/222, Train_loss: 0.0699
Train_dice: 0.9363
8/222, Train_loss: 0.0755
Train_dice: 0.9338
9/222, Train_loss: 0.0700
Train_dice: 0.9391
10/222, Train_loss: 0.0706
Train_dice: 0.9345
11/2

## Check the Size of Data (Add to main) - For Debugging

In [ ]:
# assuming prepare function has been imported from another file
train_loader, test_loader = prepare(data_dir)

for batch_data in  train_loader:
  volume = batch_data["vol"]
  label = batch_data["seg"]

  # print size of volume and label tensors
  print(f"Volume size: {volume.size()}")
  print(f"Label size: {label.size()}")

  # only process one batch
  break